# LSTM v2 ハイパーパラメータ最適化

このノートブックでは、LSTM v2ハイブリッドモデルのハイパーパラメータをOptunaを使用して最適化します。

## 分析結果に基づく最適化戦略

### 性能低下の主要因
1. **ハイパーパラメータ最適化の欠如**（主要因）
   - LSTM v1: Optuna 20試行で最適化済み
   - LSTM v2: デフォルトパラメータのみ使用

2. **attention融合方式のエラー**（修正済み）
   - 次元不一致によるエラーを修正

3. **demographics特徴量エンジニアリングの未検証**
   - 最適化プロセスで検証

### 最適化目標
- LSTM v1性能（F1-macro: 0.5200）を上回る
- 融合方式の最適選択
- 効率的なトレーニング時間


In [3]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import json
import pickle
import warnings
warnings.filterwarnings('ignore')

# プロジェクトルートを追加
project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

# Optuna関連
import optuna
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner
import optuna.visualization as vis

# 自作モジュール
from src.lstm_v2_model import LSTMv2HybridModel
from src.lstm_v2_trainer import LSTMv2Trainer

# LSTMv2Trainerクラスをインスタンス化してデータをロード
trainer = LSTMv2Trainer()
data = trainer.load_preprocessed_data()
# 結果保存用ディレクトリ
os.makedirs('../output/experiments/lstm_v2_optimization', exist_ok=True)
os.makedirs('../output/experiments/lstm_v2_optimized', exist_ok=True)

print("ライブラリ読み込み完了")
print(f"実行時刻: {datetime.now()}")


Mixed precision enabled for better GPU performance
GPU利用可能: 1台
使用GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
LSTM v2学習環境初期化完了
実験名: lstm_v2
ウィンドウ設定: w64_s16
出力ディレクトリ: ../output/experiments/lstm_v2_w64_s16
前処理済みデータ: ../output/experiments/lstm_v2_w64_s16/preprocessed
GPU利用可能: True
前処理済みデータを読み込み中...
データ読み込み完了:
  センサーデータ: (13393, 64, 332)
  Demographics: (13393, 20)
  ラベル: (13393,)
  クラス数: 18
ライブラリ読み込み完了
実行時刻: 2025-07-07 11:35:12.469583


## データ読み込み


In [4]:
# データ読み込み
print("データ読み込み中...")
# 参考ノートブックの構造に合わせてデータ読み込み
import pickle

# LSTM v2の前処理済みデータを読み込み
config_name = "w64_s16"  # LSTM v1と同じ設定で比較
data_dir = f"../output/experiments/lstm_v2_{config_name}/preprocessed"

with open(f"{data_dir}/X_sensor_windows.pkl", "rb") as f:
    X_sensor_all = pickle.load(f)

with open(f"{data_dir}/X_demographics_windows.pkl", "rb") as f:
    X_demographics_all = pickle.load(f)

with open(f"{data_dir}/y_windows.pkl", "rb") as f:
    y_all = pickle.load(f)

with open(f"{data_dir}/label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

# 訓練・検証データの分割（8:2）
from sklearn.model_selection import train_test_split
import numpy as np

# numpyアレイに変換
X_sensor_all = np.array(X_sensor_all)
X_demographics_all = np.array(X_demographics_all)
y_all = np.array(y_all)

X_sensor_train, X_sensor_val, X_demographics_train, X_demographics_val, y_train, y_val = train_test_split(
    X_sensor_all, X_demographics_all, y_all, 
    test_size=0.2, random_state=42, stratify=y_all
)

print(f"訓練データ: センサー{np.array(X_sensor_train).shape}, demographics{np.array(X_demographics_train).shape}, ラベル{np.array(y_train).shape}")
print(f"検証データ: センサー{np.array(X_sensor_val).shape}, demographics{np.array(X_demographics_val).shape}, ラベル{np.array(y_val).shape}")

# 入力形状とクラス数
sensor_input_shape = np.array(X_sensor_train).shape[1:]
demographics_input_shape = np.array(X_demographics_train).shape[1:]
num_classes = len(np.unique(y_train))

print(f"センサー入力形状: {sensor_input_shape}")
print(f"Demographics入力形状: {demographics_input_shape}")
print(f"クラス数: {num_classes}")


データ読み込み中...
訓練データ: センサー(10714, 64, 332), demographics(10714, 20), ラベル(10714,)
検証データ: センサー(2679, 64, 332), demographics(2679, 20), ラベル(2679,)
センサー入力形状: (64, 332)
Demographics入力形状: (20,)
クラス数: 18


In [5]:
# 最適化設定
N_TRIALS = 30  # 試行回数
TIMEOUT = 7200  # 2時間のタイムアウト
STUDY_NAME = "lstm_v2_hyperopt"
STORAGE_URL = f"sqlite:///../output/experiments/lstm_v2_optimization/{STUDY_NAME}.db"

# LSTM v1ベンチマーク性能
LSTM_V1_BENCHMARK = {
    'f1_macro': 0.5200,
    'accuracy': 0.5558
}

print(f"最適化設定:")
print(f"  試行回数: {N_TRIALS}")
print(f"  タイムアウト: {TIMEOUT}秒 ({TIMEOUT//3600}時間)")
print(f"  ベンチマーク: F1-macro {LSTM_V1_BENCHMARK['f1_macro']:.4f}")


最適化設定:
  試行回数: 30
  タイムアウト: 7200秒 (2時間)
  ベンチマーク: F1-macro 0.5200


In [6]:
def objective(trial):
    """
    Optuna目的関数
    """
    # ハイパーパラメータ提案
    params = {
        # アーキテクチャパラメータ
        'lstm_units_1': trial.suggest_int('lstm_units_1', 32, 128, step=16),
        'lstm_units_2': trial.suggest_int('lstm_units_2', 16, 64, step=8),
        'dense_units': trial.suggest_int('dense_units', 16, 64, step=8),
        'demographics_dense_units': trial.suggest_int('demographics_dense_units', 8, 32, step=4),
        'fusion_dense_units': trial.suggest_int('fusion_dense_units', 16, 48, step=8),
        
        # 正則化パラメータ
        'dropout_rate': trial.suggest_float('dropout_rate', 0.1, 0.5, step=0.1),
        'dense_dropout_rate': trial.suggest_float('dense_dropout_rate', 0.1, 0.4, step=0.1),
        
        # 訓練パラメータ
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True),
        'batch_size': trial.suggest_categorical('batch_size', [16, 32, 64]),
        
        # 融合方式
        'fusion_type': trial.suggest_categorical('fusion_type', ['concatenate', 'attention', 'gated']),
        
        # 早期停止
        'patience': trial.suggest_int('patience', 10, 20),
        'reduce_lr_patience': trial.suggest_int('reduce_lr_patience', 5, 10),
        
        # 固定パラメータ
        'epochs': 50,  # 最大エポック数
        'use_tqdm': False,  # プログレスバー無効
        'use_tensorboard': False  # TensorBoard無効
    }
    
    try:
        # モデル作成
        model = LSTMv2HybridModel(
            sensor_input_shape=sensor_input_shape,
            demographics_input_shape=demographics_input_shape,
            num_classes=num_classes,
            **params
        )
        
        # モデル構築
        model.build_model()
        
        # 訓練
        history = model.train(
            X_sensor_train, X_demographics_train, y_train,
            X_sensor_val, X_demographics_val, y_val,
            verbose=0
        )
        
        # 評価
        results = model.evaluate(
            X_sensor_val, X_demographics_val, y_val
        )
        
        # F1-macroスコアを最大化
        f1_macro = results['f1_macro']
        
        # 追加メトリクス記録
        trial.set_user_attr('accuracy', results['accuracy'])
        trial.set_user_attr('f1_weighted', results['f1_weighted'])
        trial.set_user_attr('precision_macro', results['precision_macro'])
        trial.set_user_attr('recall_macro', results['recall_macro'])
        trial.set_user_attr('training_time', results.get('training_time', 0))
        
        # メモリクリア
        del model
        
        return f1_macro
        
    except Exception as e:
        print(f"Trial {trial.number} failed: {str(e)}")
        return 0.0  # 失敗時は最低スコア

print("目的関数定義完了")


目的関数定義完了


In [7]:
# Optunaスタディ作成
sampler = TPESampler(seed=42)
pruner = MedianPruner(n_startup_trials=5, n_warmup_steps=10)

study = optuna.create_study(
    direction='maximize',
    sampler=sampler,
    pruner=pruner,
    study_name=STUDY_NAME,
    storage=STORAGE_URL,
    load_if_exists=True
)

print(f"最適化開始: {datetime.now()}")
print(f"目標: F1-macro > {LSTM_V1_BENCHMARK['f1_macro']:.4f}")
print("="*50)

# 最適化実行
study.optimize(
    objective,
    n_trials=N_TRIALS,
    timeout=TIMEOUT,
    show_progress_bar=True
)

print("="*50)
print(f"最適化完了: {datetime.now()}")


[I 2025-07-07 11:35:34,602] A new study created in RDB with name: lstm_v2_hyperopt


最適化開始: 2025-07-07 11:35:34.607697
目標: F1-macro > 0.5200


  0%|          | 0/30 [00:00<?, ?it/s]

Mixed precision enabled for better GPU performance
GPU利用可能: 1台
使用GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
LSTM v2ハイブリッドモデル初期化完了
センサー入力形状: (64, 332)
Demographics入力形状: (20,)
クラス数: 18
融合方式: concatenate
GPU利用可能: True
LSTM v2ハイブリッドモデルを構築中...


I0000 00:00:1751855735.021324   17074 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5660 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050, pci bus id: 0000:01:00.0, compute capability: 8.6


...

335/335 ━━━━━━━━━━━━━━━━━━━━ 14s 41ms/step - accuracy: 0.5795 - loss: 1.3113 - val_accuracy: 0.5506 - val_loss: 1.4907 - learning_rate: 0.0012
Epoch 28/50
334/335 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.5771 - loss: 1.3205
Epoch 28: val_loss improved from 1.49069 to 1.47129, saving model to lstm_v2_hybrid_best.h5


335/335 ━━━━━━━━━━━━━━━━━━━━ 13s 39ms/step - accuracy: 0.5770 - loss: 1.3206 - val_accuracy: 0.5379 - val_loss: 1.4713 - learning_rate: 0.0012
Epoch 29/50
335/335 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.5857 - loss: 1.2722
Epoch 29: val_loss did not improve from 1.47129
335/335 ━━━━━━━━━━━━━━━━━━━━ 13s 38ms/step - accuracy: 0.5857 - loss: 1.2723 - val_accuracy: 0.5278 - val_loss: 1.5128 - learning_rate: 0.0012
Epoch 30/50
335/335 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.6054 - loss: 1.2301
Epoch 30: val_loss did not improve from 1.47129
335/335 ━━━━━━━━━━━━━━━━━━━━ 13s 38ms/step - accuracy: 0.6053 - loss: 1.2303 - val_accuracy: 0.5390 - val_loss: 1.5061 - learning_rate: 0.0012
Epoch 31/50
334/335 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.5977 - loss: 1.2673
Epoch 31: val_loss did not improve from 1.47129
335/335 ━━━━━━━━━━━━━━━━━━━━ 13s 39ms/step - accuracy: 0.5978 - loss: 1.2672 - val_accuracy: 0.5267 - val_loss: 1.5421 - learning_rate: 0.0012
Epoch 32/50
335/335 

335/335 ━━━━━━━━━━━━━━━━━━━━ 13s 40ms/step - accuracy: 0.6035 - loss: 1.2460 - val_accuracy: 0.5483 - val_loss: 1.4710 - learning_rate: 0.0012
Epoch 34/50
334/335 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.6264 - loss: 1.1927
Epoch 34: val_loss did not improve from 1.47103
335/335 ━━━━━━━━━━━━━━━━━━━━ 13s 40ms/step - accuracy: 0.6264 - loss: 1.1930 - val_accuracy: 0.5349 - val_loss: 1.5189 - learning_rate: 0.0012
Epoch 35/50
334/335 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.6101 - loss: 1.2397
Epoch 35: val_loss improved from 1.47103 to 1.44464, saving model to lstm_v2_hybrid_best.h5


335/335 ━━━━━━━━━━━━━━━━━━━━ 14s 41ms/step - accuracy: 0.6101 - loss: 1.2397 - val_accuracy: 0.5513 - val_loss: 1.4446 - learning_rate: 0.0012
Epoch 36/50
335/335 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.6182 - loss: 1.2063
Epoch 36: val_loss did not improve from 1.44464
335/335 ━━━━━━━━━━━━━━━━━━━━ 14s 41ms/step - accuracy: 0.6181 - loss: 1.2063 - val_accuracy: 0.5375 - val_loss: 1.5066 - learning_rate: 0.0012
Epoch 37/50
335/335 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.6252 - loss: 1.2181
Epoch 37: val_loss did not improve from 1.44464
335/335 ━━━━━━━━━━━━━━━━━━━━ 14s 41ms/step - accuracy: 0.6252 - loss: 1.2182 - val_accuracy: 0.5558 - val_loss: 1.4815 - learning_rate: 0.0012
Epoch 38/50
335/335 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.6306 - loss: 1.1601
Epoch 38: val_loss did not improve from 1.44464
335/335 ━━━━━━━━━━━━━━━━━━━━ 14s 41ms/step - accuracy: 0.6305 - loss: 1.1602 - val_accuracy: 0.5610 - val_loss: 1.4705 - learning_rate: 0.0012
Epoch 39/50
334/335 

335/335 ━━━━━━━━━━━━━━━━━━━━ 14s 40ms/step - accuracy: 0.6838 - loss: 1.0853 - val_accuracy: 0.6002 - val_loss: 1.4138 - learning_rate: 5.9931e-04
Epoch 47/50
334/335 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.6906 - loss: 1.0238
Epoch 47: val_loss did not improve from 1.41384
335/335 ━━━━━━━━━━━━━━━━━━━━ 14s 41ms/step - accuracy: 0.6907 - loss: 1.0237 - val_accuracy: 0.6032 - val_loss: 1.4493 - learning_rate: 5.9931e-04
Epoch 48/50
334/335 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.7006 - loss: 0.9838
Epoch 48: val_loss did not improve from 1.41384
335/335 ━━━━━━━━━━━━━━━━━━━━ 14s 42ms/step - accuracy: 0.7006 - loss: 0.9838 - val_accuracy: 0.5950 - val_loss: 1.4518 - learning_rate: 5.9931e-04
Epoch 49/50
334/335 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.7157 - loss: 0.9581
Epoch 49: val_loss did not improve from 1.41384
335/335 ━━━━━━━━━━━━━━━━━━━━ 14s 41ms/step - accuracy: 0.7156 - loss: 0.9582 - val_accuracy: 0.6054 - val_loss: 1.4499 - learning_rate: 5.9931e-04
Epoc

In [8]:
# 最適化結果
print("=== 最適化結果 ===")
print(f"試行回数: {len(study.trials)}")
print(f"最良スコア: {study.best_value:.4f}")
print(f"LSTM v1ベンチマーク: {LSTM_V1_BENCHMARK['f1_macro']:.4f}")
print(f"改善度: {study.best_value - LSTM_V1_BENCHMARK['f1_macro']:+.4f}")

# 最良パラメータ
print("\n=== 最良パラメータ ===")
best_params = study.best_params
for key, value in best_params.items():
    print(f"{key}: {value}")

# 最良試行の詳細
best_trial = study.best_trial
print("\n=== 最良試行の詳細 ===")
print(f"F1-macro: {best_trial.value:.4f}")
print(f"Accuracy: {best_trial.user_attrs.get('accuracy', 'N/A'):.4f}")
print(f"F1-weighted: {best_trial.user_attrs.get('f1_weighted', 'N/A'):.4f}")
print(f"Precision-macro: {best_trial.user_attrs.get('precision_macro', 'N/A'):.4f}")
print(f"Recall-macro: {best_trial.user_attrs.get('recall_macro', 'N/A'):.4f}")
print(f"Training time: {best_trial.user_attrs.get('training_time', 'N/A'):.2f}s")


=== 最適化結果 ===
試行回数: 8
最良スコア: 0.0000
LSTM v1ベンチマーク: 0.5200
改善度: -0.5200

=== 最良パラメータ ===
lstm_units_1: 64
lstm_units_2: 64
dense_units: 56
demographics_dense_units: 24
fusion_dense_units: 16
dropout_rate: 0.1
dense_dropout_rate: 0.1
learning_rate: 0.005399484409787433
batch_size: 32
fusion_type: concatenate
patience: 12
reduce_lr_patience: 6

=== 最良試行の詳細 ===
F1-macro: 0.0000


ValueError: Unknown format code 'f' for object of type 'str'

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import json
import pickle
import warnings
warnings.filterwarnings('ignore')

# プロジェクトルートを追加
project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

# Optuna関連
import optuna
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner
import optuna.visualization as vis

# 自作モジュール
from src.lstm_v2_model import LSTMv2HybridModel
from src.lstm_v2_trainer import LSTMv2Trainer

# LSTMv2Trainerクラスをインスタンス化してデータをロード
trainer = LSTMv2Trainer()
data = trainer.load_preprocessed_data()
# 結果保存用ディレクトリ
os.makedirs('../output/experiments/lstm_v2_optimization', exist_ok=True)
os.makedirs('../output/experiments/lstm_v2_optimized', exist_ok=True)

print("ライブラリ読み込み完了")
print(f"実行時刻: {datetime.now()}")


Mixed precision enabled for better GPU performance
GPU利用可能: 1台
使用GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
LSTM v2学習環境初期化完了
実験名: lstm_v2
ウィンドウ設定: w64_s16
出力ディレクトリ: ../output/experiments/lstm_v2_w64_s16
前処理済みデータ: ../output/experiments/lstm_v2_w64_s16/preprocessed
GPU利用可能: True
前処理済みデータを読み込み中...
データ読み込み完了:
  センサーデータ: (13393, 64, 332)
  Demographics: (13393, 20)
  ラベル: (13393,)
  クラス数: 18
ライブラリ読み込み完了
実行時刻: 2025-07-07 11:35:12.469583


In [ ]:
# 融合方式別の性能分析
fusion_results = {}
for trial in study.trials:
    if trial.state == optuna.trial.TrialState.COMPLETE:
        fusion_type = trial.params.get('fusion_type', 'unknown')
        if fusion_type not in fusion_results:
            fusion_results[fusion_type] = []
        fusion_results[fusion_type].append(trial.value)

# 融合方式別統計
print("\n=== 融合方式別性能 ===")
fusion_stats = {}
for fusion_type, scores in fusion_results.items():
    if scores:
        stats = {
            'count': len(scores),
            'mean': np.mean(scores),
            'std': np.std(scores),
            'max': np.max(scores),
            'min': np.min(scores)
        }
        fusion_stats[fusion_type] = stats
        print(f"{fusion_type}: {stats['mean']:.4f}±{stats['std']:.4f} (max: {stats['max']:.4f}, n={stats['count']})")

# 最良融合方式
best_fusion_type = max(fusion_stats.keys(), key=lambda x: fusion_stats[x]['max'])
print(f"\n最良融合方式: {best_fusion_type} (max F1-macro: {fusion_stats[best_fusion_type]['max']:.4f})")


In [ ]:
# 最適化履歴の可視化
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. 最適化履歴
ax1 = axes[0, 0]
optuna.visualization.matplotlib.plot_optimization_history(study, ax=ax1)
ax1.axhline(y=LSTM_V1_BENCHMARK['f1_macro'], color='red', linestyle='--', 
            label=f'LSTM v1 Benchmark ({LSTM_V1_BENCHMARK["f1_macro"]:.4f})')
ax1.legend()
ax1.set_title('最適化履歴')

# 2. パラメータ重要度
ax2 = axes[0, 1]
try:
    optuna.visualization.matplotlib.plot_param_importances(study, ax=ax2)
    ax2.set_title('パラメータ重要度')
except:
    ax2.text(0.5, 0.5, 'パラメータ重要度\n計算不可', ha='center', va='center')
    ax2.set_title('パラメータ重要度')

# 3. 融合方式別性能
ax3 = axes[1, 0]
fusion_names = list(fusion_results.keys())
fusion_scores = [fusion_results[name] for name in fusion_names]
ax3.boxplot(fusion_scores, labels=fusion_names)
ax3.axhline(y=LSTM_V1_BENCHMARK['f1_macro'], color='red', linestyle='--', 
            label=f'LSTM v1 Benchmark')
ax3.set_ylabel('F1-macro Score')
ax3.set_title('融合方式別性能')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. 試行回数 vs 性能
ax4 = axes[1, 1]
trial_numbers = [trial.number for trial in study.trials if trial.state == optuna.trial.TrialState.COMPLETE]
trial_values = [trial.value for trial in study.trials if trial.state == optuna.trial.TrialState.COMPLETE]
ax4.scatter(trial_numbers, trial_values, alpha=0.6)
ax4.axhline(y=LSTM_V1_BENCHMARK['f1_macro'], color='red', linestyle='--', 
            label=f'LSTM v1 Benchmark')
ax4.set_xlabel('Trial Number')
ax4.set_ylabel('F1-macro Score')
ax4.set_title('試行回数 vs 性能')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../output/experiments/lstm_v2_optimization/optimization_analysis.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# 最適パラメータでモデル再構築・評価
print("最適モデルの詳細評価を実行中...")

# 最適パラメータでモデル作成
optimal_model = LSTMv2HybridModel(
    sensor_input_shape=sensor_input_shape,
    demographics_input_shape=demographics_input_shape,
    num_classes=num_classes,
    **best_params,
    epochs=50,
    use_tqdm=True,
    use_tensorboard=True
)

# モデル構築
optimal_model.build_model()

# 訓練
model_save_path = '../output/experiments/lstm_v2_optimized/best_model.h5'
history = optimal_model.train(
    X_sensor_train, X_demographics_train, y_train,
    X_sensor_val, X_demographics_val, y_val,
    model_save_path=model_save_path
)

# 詳細評価
results = optimal_model.evaluate(
    X_sensor_val, X_demographics_val, y_val
)

print("\n=== 最適モデル詳細評価結果 ===")
print(f"F1-macro: {results['f1_macro']:.4f}")
print(f"Accuracy: {results['accuracy']:.4f}")
print(f"F1-weighted: {results['f1_weighted']:.4f}")
print(f"Precision-macro: {results['precision_macro']:.4f}")
print(f"Recall-macro: {results['recall_macro']:.4f}")

# LSTM v1との比較
print("\n=== LSTM v1との比較 ===")
print(f"LSTM v1 F1-macro: {LSTM_V1_BENCHMARK['f1_macro']:.4f}")
print(f"LSTM v2 F1-macro: {results['f1_macro']:.4f}")
print(f"改善度: {results['f1_macro'] - LSTM_V1_BENCHMARK['f1_macro']:+.4f}")
print(f"改善率: {(results['f1_macro'] / LSTM_V1_BENCHMARK['f1_macro'] - 1) * 100:+.2f}%")


In [ ]:
# 最適化結果の保存
optimization_results = {
    'timestamp': datetime.now().isoformat(),
    'n_trials': len(study.trials),
    'best_value': study.best_value,
    'best_params': study.best_params,
    'best_trial_attrs': best_trial.user_attrs,
    'lstm_v1_benchmark': LSTM_V1_BENCHMARK,
    'improvement': study.best_value - LSTM_V1_BENCHMARK['f1_macro'],
    'improvement_rate': (study.best_value / LSTM_V1_BENCHMARK['f1_macro'] - 1) * 100,
    'fusion_stats': fusion_stats,
    'best_fusion_type': best_fusion_type,
    'final_evaluation': results
}

# JSON保存
with open('../output/experiments/lstm_v2_optimization/optimization_results.json', 'w', encoding='utf-8') as f:
    json.dump(optimization_results, f, indent=2, ensure_ascii=False)

# 最適パラメータ保存
with open('../output/experiments/lstm_v2_optimization/best_params.json', 'w', encoding='utf-8') as f:
    json.dump(best_params, f, indent=2, ensure_ascii=False)

# 訓練履歴保存
optimal_model.save_training_history('../output/experiments/lstm_v2_optimization/training_history.json')

# モデル設定保存
model_config = optimal_model.get_config()
with open('../output/experiments/lstm_v2_optimization/model_config.json', 'w', encoding='utf-8') as f:
    json.dump(model_config, f, indent=2, ensure_ascii=False)

print("結果保存完了:")
print(f"  - 最適化結果: ../output/experiments/lstm_v2_optimization/optimization_results.json")
print(f"  - 最適パラメータ: ../output/experiments/lstm_v2_optimization/best_params.json")
print(f"  - 訓練履歴: ../output/experiments/lstm_v2_optimization/training_history.json")
print(f"  - モデル設定: ../output/experiments/lstm_v2_optimization/model_config.json")
print(f"  - 最適モデル: {model_save_path}")


In [ ]:
print("\n" + "="*60)
print("LSTM v2 ハイパーパラメータ最適化 完了")
print("="*60)

print(f"\n【最適化結果】")
print(f"  試行回数: {len(study.trials)}")
print(f"  最良F1-macro: {study.best_value:.4f}")
print(f"  LSTM v1ベンチマーク: {LSTM_V1_BENCHMARK['f1_macro']:.4f}")
print(f"  改善度: {study.best_value - LSTM_V1_BENCHMARK['f1_macro']:+.4f}")
print(f"  改善率: {(study.best_value / LSTM_V1_BENCHMARK['f1_macro'] - 1) * 100:+.2f}%")

print(f"\n【最適融合方式】")
print(f"  融合方式: {best_fusion_type}")
print(f"  最高性能: {fusion_stats[best_fusion_type]['max']:.4f}")

print(f"\n【主要パラメータ】")
key_params = ['fusion_type', 'lstm_units_1', 'lstm_units_2', 'learning_rate', 'dropout_rate']
for param in key_params:
    if param in best_params:
        print(f"  {param}: {best_params[param]}")

# 成功判定
success = study.best_value > LSTM_V1_BENCHMARK['f1_macro']
print(f"\n【最適化成功】: {'✓' if success else '✗'}")

if success:
    print("\n✓ LSTM v2がLSTM v1の性能を上回りました！")
    print("✓ Demographics情報の統合が効果的でした")
    print("✓ ハイパーパラメータ最適化により大幅な性能向上を達成")
else:
    print("\n⚠ LSTM v2がLSTM v1の性能に届きませんでした")
    print("  さらなる最適化が必要です")

print(f"\n実行時刻: {datetime.now()}")
print("="*60)
